To detect movements in a video, we use background subtraction. There are many different background subtraction algorithms, and each algorithm has various parameters. You can use the default parameter settings in Birdwatcher, but you can also modify the parameters and optimize these for your own videos.

This notebook can be used to play around with all options and various parameters settings of background subtraction algorithms when using Birdwatcher for movement detection, and see how this influences the results.

In [ ]:
import itertools
import numpy as np
import pandas as pd
import seaborn as sns

import birdwatcher as bw
from birdwatcher.plotting import imshow_frame # birdwatcher has vizualization tools
%matplotlib inline

### Select video fragment

In [ ]:
vfs = bw.VideoFileStream(r'..\videos\zebrafinch.MTS')

Choose a short representative video fragment where the object of interest is moving quite a lot.

In [ ]:
starttime = '00:02:10'   # in HOURS:MM:SS
duration = 20            # seconds

vfs_fragment = (vfs.iter_frames(startat=starttime, nframes=vfs.avgframerate*duration)
                .tovideo('output/videofragment.MTS', framerate=vfs.avgframerate))

In [ ]:
# or load video fragment directly from disk
vfs_fragment = bw.VideoFileStream(r'output/videofragment.MTS')

In [ ]:
# vfs_fragment.show(framerate=150)

### Choose parameters

In this example, we will use background subtraction MOG2, and try to find parameter values for this algorithm. Also some manipulations before or after performing background subtraction might improve location detection, and therefore, you can also determine the settings of those.

First, decide which settings you would like, by adding various values in the list after each parameter. 

In [ ]:
bgs_params = {'History': [4, 8, 12],
              'ComplexityReductionThreshold': [0.05],
              'BackgroundRatio': [0.1],
              'NMixtures': [7],
              'VarInit': [15],
              'VarMin': [4],
              'VarMax': [75],
              'VarThreshold': [10],
              'VarThresholdGen': [9],
              'DetectShadows': [False],
              'ShadowThreshold': [0.5],
              'ShadowValue': [127]}

other_settings = {'color': [True, False],   # booleans only
                  'resizebyfactor': [1, (2/3), (1/3)],   # use '1' for no change in size
                  'blur': [1, 5, 10],   # use '1' for no blur
                  'morphologyex': [True, False]}   # booleans only}

In [ ]:
def product_dict(**kwargs):
    keys = kwargs.keys()
    vals = kwargs.values()
    for instance in itertools.product(*vals):
        yield dict(zip(keys, instance))

all_combinations = list(product_dict(**bgs_params, **other_settings))
print(f'There are {len(all_combinations)} different combinations of settings to perform movement detection.')

The higher the number of combinations, the longer the next step (running movement detection) will take. Another option is to start by tweaking some parameters with larger steps of parameter values, and fine-tune the values in next rounds.

### Run movemement detection per combination of settings

Movement detection is done for each combination of settings, and the mean coordinate per frame is saved in a Pandas dataframe.

**WARNING:** This step might take a while, depending on the number of settings combinations!

In [ ]:
%%time
def movement_per_setting(vfs, bgs_params, other_settings):
    
    list_with_dfs = []

    for settings in product_dict(**bgs_params, **other_settings):
        
        frames = vfs.iter_frames(color=settings['color'])
        
        if settings['resizebyfactor'] != 1:
            val = settings['resizebyfactor']
            frames = frames.resizebyfactor(val,val)
        
        if settings['blur'] != 1:
            val = settings['blur']
            frames = frames.blur((val,val))
        
        # extract bgs settings and apply bgs
        params = {p:settings[p] for p in bgs_params.keys()}  
        bgs = bw.BackgroundSubtractorMOG2(**params)
        frames = frames.apply_backgroundsegmenter(bgs, learningRate=-1)
        
        if settings['morphologyex']:
            frames = frames.morphologyex(morphtype='open', kernelsize=2)
        
        # find mean of nonzero coordinates
        coordinates = frames.find_nonzero()
        coordsmean = np.array([c.mean(0) if c.size>0 else (np.nan, np.nan) for c in coordinates])

        # save coordsmean x,y in pandas DataFrame with associated settings as column labels
        settings['coords'] = ['x', 'y']
        columns = pd.MultiIndex.from_frame(pd.DataFrame(settings))
        df = pd.DataFrame(coordsmean, columns=columns)
        list_with_dfs.append(df)

    df = pd.concat(list_with_dfs, axis=1)
    return df

df = movement_per_setting(vfs_fragment, bgs_params, other_settings)

In [ ]:
df

Here, you see in the column index each combination of settings that is used to run movement detection. The rows represent the resulted mean x,y coordinates per frame. (NaN means there were no nonzero pixels found for that frame)

### Long-format dataframe

For plotting it is easier to organize the data in a long-format.

In [ ]:
# create long-format
df.index.name = 'framenumber'
df_long = df.stack(list(range(df.columns.nlevels)), dropna=False).reset_index()  # stack all column level
df_long = df_long.rename({0: 'pixel'}, axis=1)
df_long

### Save/load Dataframe

Save the results as csv file, to be able to load the dataframe later without running all movement detections per setting combination again!

In [ ]:
# save DataFrame
df_long.to_csv('output/dataframe_params.csv', index=False)

In [ ]:
# load DataFrame
df_long = pd.read_csv('output/dataframe_params.csv', engine='python')

In [ ]:
df_long.head()

### Correction resizebyfactor

Setting 'resizebyfactor' changes the width and height of the frames. Below, we correct for this change in pixel resolution, so that it's easier to see and compare the effects of different settings on the movementdetection results below.

In [ ]:
df_long['pixel'] = df_long['pixel'] / df_long['resizebyfactor']
df_long.head()

### Visualize results

In [ ]:
def select_data(df, **kwargs):
    
    for key, value in kwargs.items():
        df = df.loc[df[key]==value]
        
    return df

Before visualizing the results, look at which settings you could compare.

In [ ]:
# the following settings have been used for backgroundsubstraction in this dataframe
all_settings = {k:list(df_long[k].unique()) for k in {**bgs_params, **other_settings}.keys()}
all_settings

In [ ]:
# the following settings have been tested with multiple values
settings = {k:all_settings[k] for k in all_settings.keys() if len(all_settings[k])>1}
settings

Here, you see for which settings multiple values have been used to run movement detection. So, these are also the settings that are interesting to compare in plots or superimpose on the video fragment: 

#### Plots

In [ ]:
# choose 2 or 3 settings you would like to compare:
rows = 'resizebyfactor'
cols = 'History'
style = None

# choose for the other settings with multiple values, which value you would like to use for this comparison:
other_values = {'color': True,
                'blur': 1,
                'morphologyex': True}

df_selection = select_data(df_long, **other_values)

# plot with seaborn
sns.relplot(x="framenumber", y="pixel", hue="coords", style=style, 
            col=cols, row=rows, kind="line", data=df_selection, 
            height=3, aspect=2)

#### Superimpose on video

In [ ]:
# colors in BGR.
colors = [('blue', (255, 0, 0)),
          ('orange', (0, 100, 255)),
          ('red', (0, 0, 255)),
          ('lime', (0, 255, 0))]

You can add other colors or change the order of colors in the list. Note that the number of colors defined, is also the maximum number of circles you can draw in the video.

In [ ]:
# choose which settings to compare
compare = {'History': [4, 8, 12]}

# choose for the other settings with multiple values, which value you would like to use for this comparison
other_values = {'color': True,
                'resizebyfactor': 1,
                'blur': 1,
                'morphologyex': True}

df_selection = select_data(df_long, **other_values)

all_combinations = list(product_dict(**compare))
print(f'There are {len(all_combinations)} combinations of settings to superimpose on a video.')

In [ ]:
def draw_multiple_circles(frames, df, compare, colors):
    
    for i, settings in enumerate(product_dict(**compare)):
        colorname = colors[i][0]
        colorcode = colors[i][1]
        print(f"The {colorname} circle has settings: {settings}")

        # select data
        df = select_data(df_selection, **settings)

        # transform into iterable
        iterdata = df.set_index(['framenumber', 'coords']).loc[:, 'pixel'].unstack().values

        frames = frames.draw_circles(iterdata, radius=60, color=colorcode)
    
    return frames


frames = vfs_fragment.iter_frames().draw_framenumbers()
frames = draw_multiple_circles(frames, df_selection, compare, colors)

# save video with circles superimposed
# vfs_circles = frames.tovideo('output/zf20s_high_multicircles.MTS', framerate=vfs.avgframerate)

In [ ]:
frames.show(framerate=100)